In [197]:
import pandas as pd
import altair as alt
from urllib.error import HTTPError

In [198]:
def claim(claim_hash, cols_claim, data_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    return df_claim

In [199]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [200]:
user_stats = '1'
airdrop_claims = '2'
lba_deposits = '3'

In [201]:
cols_claim = {
    user_stats : ['SENDER', 'DURATION', 'AMOUNT'],
    airdrop_claims : ['SENDER','AMOUNT','TIME'],
    lba_deposits : ['SENDER','AMOUNT','DENOM','ACTION','TIME']
}

In [202]:
data_claim = {
    user_stats : [['user1_1',3,10],
                  ['user1_2',3,3],
                  ['user1_3',3,4],
                  ['user1_4',3,67],
                  ['user1_5',3,33],
                 ['user1',9,20],
                 ['user1',18,15],
                 ['user2',3,10],
                 ['user2',6,120],
                 ['user2',18,13],
                 ['user2',3,10],
                 ['user3',6,120],
                 ['user3',18,13]],
    airdrop_claims :
        [
            ['user1',300,'2021-09-21T07:00:00Z'],
            ['user2',70,'2021-09-21T07:00:00Z'],
            ['user3',34,'2021-09-21T07:00:00Z'],
            ['user1_1',132,'2021-09-21T07:00:00Z'],
            ['user4',132,'2021-09-21T07:00:00Z']
        ],
    lba_deposits : [['user1_1',20,'MARS','deposit','2021-09-21T08:00:00Z'],
                    ['user1_1',50,'UST','deposit','2021-09-21T08:00:00Z'],
                    ['user2',70,'MARS','deposit','2021-09-21T08:00:00Z'],
                    ['user2',80,'UST','deposit','2021-09-21T08:00:00Z'],
                    ['user2',10,'UST','withdraw','2021-09-21T09:00:00Z'],
                    ['user1',70,'MARS','deposit','2021-09-21T08:00:00Z'],
                    ['user4',132,'MARS','deposit','2021-09-21T09:00:00Z']]}

### Data

**User stats**

In [203]:
user_stats_df = claim(user_stats,cols_claim,data_claim)
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,sender,duration,amount
0,user1_1,3,10
1,user1_2,3,3
2,user1_3,3,4
3,user1_4,3,67
4,user1_5,3,33
5,user1,9,20
6,user1,18,15
7,user2,3,10
8,user2,6,120
9,user2,18,13


**Airdrop**

In [204]:
airdrop_claims_df = claim(airdrop_claims,cols_claim,data_claim)
airdrop_claims_df.columns = [c.lower() for c in airdrop_claims_df.columns]
airdrop_claims_df

,sender,amount,time
0,user1,300,2021-09-21T07:00:00Z
1,user2,70,2021-09-21T07:00:00Z
2,user3,34,2021-09-21T07:00:00Z
3,user1_1,132,2021-09-21T07:00:00Z
4,user4,132,2021-09-21T07:00:00Z


**LBA deposits**

In [205]:
lba_deposits_df = claim(lba_deposits,cols_claim,data_claim)
lba_deposits_df.columns = [c.lower() for c in lba_deposits_df.columns]
lba_deposits_df['time'] = pd.to_datetime(lba_deposits_df.time)
lba_deposits_df['amount'] = lba_deposits_df.apply(lambda row: -row.amount if row.action=='withdraw' else row.amount, axis=1)
lba_deposits_df

,sender,amount,denom,action,time
0,user1_1,20,MARS,deposit,2021-09-21 08:00:00+00:00
1,user1_1,50,UST,deposit,2021-09-21 08:00:00+00:00
2,user2,70,MARS,deposit,2021-09-21 08:00:00+00:00
3,user2,80,UST,deposit,2021-09-21 08:00:00+00:00
4,user2,-10,UST,withdraw,2021-09-21 09:00:00+00:00
5,user1,70,MARS,deposit,2021-09-21 08:00:00+00:00
6,user4,132,MARS,deposit,2021-09-21 09:00:00+00:00


**Hourly Airdrops**

In [206]:
lba_deposits_df['hour'] = lba_deposits_df.time.dt.strftime("%Y-%m-%d %H:00")

In [207]:
mars = lba_deposits_df[lba_deposits_df.denom=='MARS']
ust = lba_deposits_df[lba_deposits_df.denom=='UST']

In [208]:
df_mars = mars.groupby(['denom','hour']).amount.sum().reset_index().sort_values(by='hour')
df_mars['cumsum'] = df_mars.amount.cumsum()
df_ust = ust.groupby(['denom','hour']).amount.sum().reset_index().sort_values(by='hour')
df_ust['cumsum'] = df_ust.amount.cumsum()
lba_deposits_hourly_df = df_ust.append(df_mars)
lba_deposits_hourly_df

C:\Users\aincerti\AppData\Local\Temp\ipykernel_6020\1845166223.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lba_deposits_hourly_df = df_ust.append(df_mars)


,denom,hour,amount,cumsum
0,UST,2021-09-21 08:00,130,130
1,UST,2021-09-21 09:00,-10,120
0,MARS,2021-09-21 08:00,160,160
1,MARS,2021-09-21 09:00,132,292


**Users partecipation**

In [209]:
aidrop_users = airdrop_claims_df.sender.unique()

In [210]:
aidrop_users_df = pd.DataFrame(aidrop_users, columns=['sender'])
aidrop_users_df['type'] = 'Airdrop'

In [211]:
p1_users = user_stats_df.sender.unique()
p1_users_df = pd.DataFrame(p1_users, columns=['sender'])
p1_users_df['type'] = 'Phase1'

In [212]:
users_type = aidrop_users_df.append(p1_users_df)
users_type = users_type.merge(users_type.groupby('sender').type.count().rename('n_types').reset_index(), on=['sender'], how='left')
users_type['new_type'] = users_type.apply(lambda row: row.type if row.n_types==1 else 'Airdrop/Phase1', axis=1)
users_type = users_type[["sender","new_type"]].drop_duplicates()
users_type

C:\Users\aincerti\AppData\Local\Temp\ipykernel_6020\669479942.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_type = aidrop_users_df.append(p1_users_df)


,sender,new_type
0,user1,Airdrop/Phase1
1,user2,Airdrop/Phase1
2,user3,Airdrop/Phase1
3,user1_1,Airdrop/Phase1
4,user4,Airdrop
6,user1_2,Phase1
7,user1_3,Phase1
8,user1_4,Phase1
9,user1_5,Phase1


**LBA Mars origin**

In [213]:
mars_source = users_type.merge(lba_deposits_df[lba_deposits_df.denom=='MARS'], on='sender').groupby('new_type').amount.sum()
mars_source

new_type
Airdrop           132
Airdrop/Phase1    160
Name: amount, dtype: int64

**Metrics**

In [214]:
users_aidrop_eligible = 100
perc_airdrop_eligible = len(airdrop_claims_df.sender.unique())/users_aidrop_eligible
perc_airdrop_eligible

0.05

In [215]:
mars_total = 1000
mars = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='MARS']
act_mars_lba = mars[mars.hour == mars.hour.max()]["cumsum"].values[0]
perc_mars_in_lba = act_mars_lba/mars_total
perc_mars_in_lba

0.292

In [216]:
usts = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='UST']
act_usts_lba = usts[usts.hour == mars.hour.max()]["cumsum"].values[0]
act_price = act_mars_lba/act_usts_lba
act_price

2.433333333333333

In [217]:
users_p1 = len(user_stats_df.sender.unique())
users_p1_lba = len(set(user_stats_df.sender.unique()).intersection(set(lba_deposits_df.sender.unique())))
perc_p1_lba = users_p1_lba/users_p1
perc_p1_lba

0.375

**P1 MARS rewards**

In [218]:
boost = pd.DataFrame([1,2.8,5.2,8,11.2,14.7],columns=['boost'],index=[3,6,9,12,15,18])

In [219]:
boost = user_stats_df.groupby('duration').sum().join(boost)
boost['amount_boosted'] = boost.amount * boost.boost

In [220]:
boost['amount_boosted_mars'] = (boost['amount_boosted']/boost['amount_boosted'].sum())*50000000

In [221]:
boost['amount_per_ust'] = boost['amount_boosted_mars']/boost.amount
boost

,amount,boost,amount_boosted,amount_boosted_mars,amount_per_ust
duration,,,,,
3,137,1.0,137.0,4.519364e+06,32988.058323
6,240,2.8,672.0,2.216798e+07,92366.563304
9,20,5.2,104.0,3.430758e+06,171537.903279
18,41,14.7,602.7,1.988190e+07,484924.457346


In [222]:
user_stats_df = user_stats_df.merge(boost.reset_index()[['duration','amount_per_ust']], on=['duration'])
user_stats_df['mars'] = user_stats_df.amount * user_stats_df.amount_per_ust
user_stats_df

,sender,duration,amount,amount_per_ust,mars
0,user1_1,3,10,32988.058323,3.298806e+05
1,user1_2,3,3,32988.058323,9.896417e+04
2,user1_3,3,4,32988.058323,1.319522e+05
3,user1_4,3,67,32988.058323,2.210200e+06
4,user1_5,3,33,32988.058323,1.088606e+06
5,user2,3,10,32988.058323,3.298806e+05
6,user2,3,10,32988.058323,3.298806e+05
7,user1,9,20,171537.903279,3.430758e+06
8,user1,18,15,484924.457346,7.273867e+06
9,user2,18,13,484924.457346,6.304018e+06


**% il LBA from P1 for each user**

In [228]:
user_p1_mars = user_stats_df.groupby('sender').mars.sum()

In [240]:
user_p1_perc_mars = lba_deposits_df[lba_deposits_df.denom=='MARS'].groupby('sender').sum().join(user_p1_mars).fillna(0)
user_p1_perc_mars['perc_p1_mars_lba'] = user_p1_perc_mars.apply(lambda row: 0 if ((row.mars == 0) or (row.amount==0)) else row.amount / row.mars,axis=1)
user_p1_perc_mars

,amount,mars,perc_p1_mars_lba
sender,,,
user1,70,1.070462e+07,0.000007
user1_1,20,3.298806e+05,0.000061
user2,70,1.804777e+07,0.000004
user4,132,0.000000e+00,0.000000


### Charts

In [359]:
users_balance_df['url'] = 'https://finder.extraterrestrial.money/mainnet/address/'+users_balance_df['sender']
if(len(users_balance_df)>5000):
    df = users_balance_df.sample(n=5000, random_state=1)
else:
    df = users_balance_df
dep_dist_balance_chart =alt.Chart(df).mark_point(opacity=1, filled=True).encode(
y=alt.Y("amnt_sum:Q",scale=alt.Scale(domain=(0, 200))),
x=alt.X("balance:Q",scale=alt.Scale(domain=(0, 200))),
href='url:N',
color=alt.Color('weighted_avg_dur',
    scale=alt.Scale(scheme='redpurple')),
tooltip=['sender', 'amnt_sum','balance','weighted_avg_dur']
).configure_view(strokeOpacity=0).interactive()
dep_dist_balance_chart

alt.Chart(...)

In [360]:
pie_ust_chart = alt.Chart(last_duration_amount).mark_bar().encode(
    y=alt.Y(field="UST deposited", type="quantitative"),
    x=alt.X(field="Lockup period", type="nominal"),
    color=alt.Color("Lockup period",
            sort=['3 months','6 months',
                              '9 months','12 months',
                              '15 months','18 months'],
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-40,
            direction='vertical')),
    tooltip=["UST deposited","Lockup period"]
).configure_view(strokeOpacity=0)
pie_ust_chart

alt.Chart(...)

In [84]:
df = hourly_new_users_df.rename(columns={'time':'Time','cumsum_new_users':'Number of total users'})
users_over_time_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Time:T',\
        axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
    y="Number of total users:Q",
    tooltip=['Time', "Number of total users:Q"]
    ).configure_mark(
        color='#fab0ba'
    ).configure_view(strokeOpacity=0)
users_over_time_chart

alt.Chart(...)

In [85]:
df = hourly_stats_df.rename(columns={'hr':'Time','tot_txs':'Number of transactions'})
txs_over_time_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Time:T', \
            axis=alt.Axis(tickCount=10, labelAngle=0, tickBand = 'center')),
    y="Number of transactions:Q",
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),"Number of transactions:Q"]
).configure_mark(
    color='#ffde85'
).configure_view(strokeOpacity=0)
txs_over_time_chart

alt.Chart(...)

In [86]:
alt.Chart(time_duration_df.rename(columns={'hr':'Time'})).mark_line(point = True).encode(
    x='Time:T',
    y='UST deposited:Q',
    color=alt.Color('Lockup period:N', scale=alt.Scale(scheme='pastel1')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'UST deposited:Q','Lockup period:N']
)

alt.Chart(...)

In [87]:
n_tx_wallet_chart = alt.Chart(count_durations_users).mark_line(point = True, color='#7DDBD3').encode(
    y=alt.Y('Number of users:Q', sort="ascending"),
    x="Number of lockup durations:O",
    tooltip=['Number of users:Q',"Number of lockup durations:Q"]
).properties(height=300).configure_view(strokeOpacity=0)
n_tx_wallet_chart

alt.Chart(...)

In [88]:
user_stats_df

,sender,duration,amount,dur_amount
0,user1_1,3,10,30
1,user1_2,3,3,9
2,user1_3,3,4,12
3,user1_4,3,67,201
4,user1_5,3,33,99
5,user1,9,20,180
6,user1,18,15,270
7,user2,3,10,30
8,user2,6,120,720
9,user2,18,13,234


In [93]:
alt.Chart(user_stats_df.rename(columns={'duration':'Lockup period (months)','amount':'Amount UST deposited'})).mark_boxplot(extent='min-max').encode(
    x=alt.X(field="Lockup period (months)", type="nominal", axis=alt.Axis(labelAngle=0),
                                    sort=[3,6,9,12,15,18]),
    y='Amount UST deposited:Q',
    color=alt.Color(field="Lockup period (months)", type="nominal",
                                    sort=['3 months','6 months',
                                          '9 months','12 months',
                                          '15 months','18 months'],
                                    scale=alt.Scale(scheme='lightorange'),
                                    legend=None),
    tooltip=['Lockup period (months)','Amount UST deposited']
)

alt.Chart(...)